In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
from IPython.display import display
##
import itertools
import json
import re
import numpy           as np
import pandas          as pd
import statsmodels.api as sm
import ipywidgets      as widgets
##
import hschain.logs  as Log
import hschain.plot  as plot
import hschain.splot as splot
## Shut up matplotlib
plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
class Tabs(object):
    def __init__(self):
        self.tab = widgets.Tab()
        display(self.tab)
    def __call__(self, name):
        out = widgets.Output()
        self.tab.children = list(self.tab.children) + [out]
        self.tab.set_title(len(self.tab.children) - 1, name)
        return out

In [ ]:
logs = Log.LogSet('../logs/')
p    = plot.CommitData(logs)

# Block and commit information

In [ ]:
tabs = Tabs()
with tabs("Height"):
    with plot.LegendPlot() as ax:
        p.plot_points(ax, reltime=True)
with tabs("Residuals"):
    with plot.LegendPlot() as ax:
        p.plot_residuals_TvsH(ax)
with tabs("Round"):
    plot.plot_round(logs)
with tabs("Round distr"):
    plot.plot_round_distr(logs)

In [ ]:
tabs = Tabs()
with tabs("Block size"):
    with plot.SimplePlot() as ax:
        p.plot_ntx(ax)
with tabs("Block size (hist)"):
    with plot.SimplePlot() as ax:
        p.plot_ntx_distr(ax)
with tabs("N sigs"):
    with plot.SimplePlot() as ax:
        p.plot_n_signatures(ax)

# Mempool

In [ ]:
df    = logs.some()
tMean = np.average(df.mempoolStat['fltT'])
tMed  = np.median(df.mempoolStat['fltT'])
#
tabs = Tabs()
with tabs("mempool size"):
    plot.plot_mempool_size(logs)
with tabs("Filtering time"):
    plt.figure()
    plt.grid()
    plt.title("Filtering time [μ=%.0fms, med=%.0fms]" % (tMean,tMed))
    plt.plot(df.mempoolStat['fltT'], '+')
with tabs("Filtering time (distr)"):
    plt.figure()
    plt.grid()
    plt.title("Filtering time [μ=%.0fms, med=%.0fms]" % (tMean,tMed))
    plt.xlabel("Time (ms)")
    plt.hist(df.mempoolStat['fltT'], bins=50)
with tabs("Filtering time per TX"):
    plt.figure()
    plt.grid()
    plt.title("Filtering time per TX")
    plt.xlabel("Time per TX (ms)")
    plt.hist(df.mempoolStat['fltT'] / df.mempoolStat['sizeB'], bins=50)

# Timing of algorithm steps

In [ ]:
splot.splot(logs, w=8096)

In [ ]:
tabs = Tabs()
for i,k in enumerate(Log.keyOrdSet):
    v = logs.some().stepsTime[k]
    with tabs(k):
        plt.figure()
        plt.grid()
        times = v['dt'].values.astype('float')/1e6
        plt.title("%s: %.1fms" % (k, np.average(times)))
        plt.hist(times, bins=100)

In [ ]:
def calc_jitter(logs,step):
    df       = pd.concat([l.consStep(step) for _,l in logs.items()])
    t0       = np.min(df['at'])
    df['at'] = (df['at'] - t0).astype('timedelta64[ms]').astype('float')
    grp = df.groupby(df.index)
    return pd.DataFrame(data={
        'max'    : grp.max()['at'],
        'min'    : grp.min()['at'],
        'median' : grp.median()['at']
    })
\
tabs = Tabs()
rs   = []
for i,s in enumerate(Log.Step):
    with tabs(s.value) :
        df   = calc_jitter(logs, s)
        r    = df['max'] - df['min']        
        #r    = df['max'] + df['min'] - 2*df['median']
        #r    = df['max'] - df['median']
        #r    = df['median'] - df['min']
        rs.append(r)
        rMin = np.min(r)
        rMax = np.max(r)
        rAvg = np.average(r)
        #
        fig,(ax1,ax2) = plt.subplots(1,2,figsize=[9, 4.8])
        #
        fig.suptitle("Jitter: %.0f / %.0f / %.0f ms" % (rMin,rAvg,rMax))
        ax1.grid()
        ax1.axhline(rAvg, color='r')
        ax1.plot(r,'+')
        #
        ax2.grid()
        ax2.axvline(rAvg, color='r')
        ax2.hist(r, bins=30)

with tabs("Boxplot"):
    fig = plt.figure()
    plt.grid()
    plt.boxplot(rs, labels=["NH","P","PV","PC","C","C'"])
    fig.savefig('optim.png')

# Gossip

In [ ]:
tabs = Tabs()
for lab in ["P","PV","PC","B"]:
    with tabs(lab):
        fig,((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=[9, 8])
        fig.suptitle("Gossip Rx/Tx%s" % lab)
        for name,df in logs.items():
            t  = df.gossip['dt']
            rx = df.gossip['Rx'+lab]
            tx = df.gossip['Tx'+lab]
            rRx = sm.OLS(rx, sm.add_constant(t)).fit()
            rTx = sm.OLS(tx, sm.add_constant(t)).fit()
            #
            ax1.grid(True)
            ax1.set_title("Rx")
            ax1.plot(t, rx)
            #
            ax2.grid(True)
            ax2.set_title('Tx')
            ax2.plot(t, tx)
            #
            ax3.grid(True)
            ax3.set_title('Excess Rx-Tx')
            ax3.plot(t, rx-tx)
            print("%8s: Rx=%-12.3g/s  Tx=%-12.3g/s" % (name, rRx.params[1], rTx.params[1]))

# Playground

In [ ]:
def group_by_HR(df):
    return {k : [d
                 for _,d
                 in v.fillna(method='ffill').groupby('R')]
            for k,v
            in df.groupby('H')}

In [ ]:
def proposal_time(df):
    t0 = df[df['msg'] == 'Entering propose']['at'].values
    t1 = df[df['msg'] == 'Making new proposal']['at'].values
    if len(t0) == 1 and len(t1) == 1:
        return (t1[0] - t0[0]).astype('timedelta64[ns]').astype('float') * 1e-6
    return None

def all_proposal_times(logs):
#    hmaps = [ [ [proposal_time(df) for df in dfseq]
#                for _,dfseq 
#                in group_by_HR(o.cons)
#              ]
#             for _,o in logs.items()
#            ]
    return hmaps

In [ ]:
#all_proposal_times(logs)
hmap  = [group_by_HR(o.cons) for _,o in logs.items()]
propT = [[proposal_time(df) for df in dfs]
         for _,dfs
         in itertools.chain(*[ h.items() for h in hmap ])
        ]
ts = list(itertools.filterfalse(lambda x : x is None, 
                           itertools.chain.from_iterable(propT)))

In [ ]:
plt.figure()
plt.grid()
plt.hist(ts, bins=30)
None